In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Helper
df = pd.read_csv("", error_bad_lines=False)
df.head()

b'Skipping line 6263115: expected 4 fields, saw 6\nSkipping line 6263116: expected 4 fields, saw 6\nSkipping line 6263117: expected 4 fields, saw 6\nSkipping line 6263118: expected 4 fields, saw 6\nSkipping line 6263119: expected 4 fields, saw 6\nSkipping line 6263120: expected 4 fields, saw 6\nSkipping line 6263121: expected 4 fields, saw 6\nSkipping line 6263122: expected 4 fields, saw 6\nSkipping line 6263123: expected 4 fields, saw 6\nSkipping line 6263124: expected 4 fields, saw 6\nSkipping line 6263125: expected 4 fields, saw 6\nSkipping line 6263126: expected 4 fields, saw 6\nSkipping line 6263127: expected 4 fields, saw 6\n'
b'Skipping line 17559410: expected 4 fields, saw 6\nSkipping line 17559411: expected 4 fields, saw 6\nSkipping line 17559412: expected 4 fields, saw 6\nSkipping line 17559413: expected 4 fields, saw 6\nSkipping line 17559414: expected 4 fields, saw 6\nSkipping line 17559415: expected 4 fields, saw 6\nSkipping line 17559416: expected 4 fields, saw 6\nSkippin

,run,tick,lifestate,name
0,1,1.0,adult,Tick 5
1,1,1.0,adult,Tick 2
2,1,1.0,adult,Tick 0
3,1,1.0,adult,Tick 3
4,1,1.0,adult,Tick 1


In [4]:
# Helper
for run in df.groupby('run'):
    df = run[1]
    print("Run {} went {} days w/ {} Ixodes".format(run[0], df['tick'].max(), df['name'].value_counts().count()))

Run 1 went 450.0 days w/ 24474 Ixode
Run 9 went 86.0 days w/ 10 Ixode
Run 17 went 450.0 days w/ 37591 Ixode
Run 25 went 450.0 days w/ 26871 Ixode
Run 33 went 86.0 days w/ 10 Ixode
Run 41 went 450.0 days w/ 26977 Ixode
Run 49 went 450.0 days w/ 81378 Ixode
Run 57 went 450.0 days w/ 48122 Ixode
Run 65 went 450.0 days w/ 17893 Ixode
Run 73 went 450.0 days w/ 29834 Ixode
Run 81 went 450.0 days w/ 14736 Ixode
Run 89 went 450.0 days w/ 29884 Ixode
Run 97 went 450.0 days w/ 40394 Ixode


In [7]:
#
# Parses parameter file and creates a dictionary that maps the habitat_suitability value with the
# corresponding run_number and host_density values
#
# { habitat: [ [run_number], [host_density] ] }
#
paramfile = ""
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").split(',')
# Try to add the values to an existing key, if that key does not exist, create it
        try:
            paramd[float(result[8])][0].append(int(result[0])) # add run number
            paramd[float(result[8])][1].append(float(result[4])) # add host density
        except KeyError:
            paramd[float(result[8])] = [[int(result[0])], [float(result[4])]]
# Helper
for k, v in paramd.items():
    print(k,v)


0.01 [[1, 11, 21, 31, 41, 51, 61, 71, 81, 91], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.02 [[2, 12, 22, 32, 42, 52, 62, 72, 82, 92], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.03 [[3, 13, 23, 33, 43, 53, 63, 73, 83, 93], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.04 [[4, 14, 24, 34, 44, 54, 64, 74, 84, 94], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.05 [[5, 15, 25, 35, 45, 55, 65, 75, 85, 95], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.06 [[6, 16, 26, 36, 46, 56, 66, 76, 86, 96], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.07 [[7, 17, 27, 37, 47, 57, 67, 77, 87, 97], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.08 [[8, 18, 28, 38, 48, 58, 68, 78, 88, 98], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.09 [[9, 19, 29, 39, 49, 59, 69, 79, 89, 99], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]
0.1 [[10, 20, 30, 40, 50, 60, 70, 80, 90, 100], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]]


In [8]:
ixode_count_dict = {}
df = pd.read_csv("", error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())

b'Skipping line 6391: expected 4 fields, saw 6\nSkipping line 6392: expected 4 fields, saw 6\nSkipping line 6393: expected 4 fields, saw 6\nSkipping line 6394: expected 4 fields, saw 6\nSkipping line 6395: expected 4 fields, saw 6\nSkipping line 6396: expected 4 fields, saw 6\nSkipping line 6397: expected 4 fields, saw 6\nSkipping line 6398: expected 4 fields, saw 6\nSkipping line 6399: expected 4 fields, saw 6\nSkipping line 6400: expected 4 fields, saw 6\nSkipping line 6401: expected 4 fields, saw 6\nSkipping line 6402: expected 4 fields, saw 6\nSkipping line 6403: expected 4 fields, saw 6\nSkipping line 13022: expected 4 fields, saw 6\nSkipping line 13023: expected 4 fields, saw 6\nSkipping line 13024: expected 4 fields, saw 6\nSkipping line 13025: expected 4 fields, saw 6\nSkipping line 13026: expected 4 fields, saw 6\nSkipping line 13027: expected 4 fields, saw 6\nSkipping line 13028: expected 4 fields, saw 6\nSkipping line 13029: expected 4 fields, saw 6\nSkipping line 13030: exp

In [9]:
# Creating a data frame from the above generated dictionary
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixode'])
df_final['host_density'] = 0
df_final.dtypes

run             int64
total_ixode     int64
host_density    int64
dtype: object

In [10]:
# Iterate through parameter dictionary and use loc to find the row with the run number we want. Once we have this, add
# the host density value for that associated run
for key, value in paramd.items():
    for i in range(len(value[0])):
        # Both of these techniques work, but loc is preferred over chained indexing
        # df_final['host_density'][df_final['run'] == value[0][i]] = value[1][i]
        df_final.loc[df_final['run']==value[0][i], 'host_density'] = value[1][i]
        df_final.loc[df_final['run']==value[0][i], 'habitat_suitability'] = key

In [14]:
matplotlib.use('Qt5Agg')
# fig = plt.figure()
for hab in df_final.groupby('habitat_suitability'):
    dat = hab[1]
    plt.plot('host_density', 'total_ixode' , data=dat, marker='o', markersize=7, linewidth=2, label=hab[0])
    plt.ylabel('Cumulative Ixodes')
    plt.xlabel('Large host density')
    plt.title("Host Density")
    plt.legend() # FIX to show values not just label
plt.show(block=True)
# plt.savefig("../data/host-density/plots/newmodel-density-adult-10ixodes-labeled.png")

